https://cookbook.openai.com/examples/assistants_api_overview_python

https://platform.openai.com/docs/assistants/overview

<img src = "https://raw.githubusercontent.com/panaverse/learn-generative-ai/4e527c5445f13cce356e2167bccca6ad2c3798be/03_chatgpt/06_assistants_code_interpreter/assistants.png">

# Importing necessary classes 

In [14]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import json
_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()


# Pretty printing Helper

In [5]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# ceating an assistant

In [15]:
import json
from openai.types.beta import Assistant

My_assistant: Assistant = client.beta.assistants.create(
    
    name = "Researcher",
    instructions = "you're a researcher, answer my question briefly, based on your research",
    model = "gpt-3.5-turbo-1106"
)

show_json(My_assistant)




{'id': 'asst_G3EEKDejKTKqiuQUIao3EwXe',
 'created_at': 1706244318,
 'description': None,
 'file_ids': [],
 'instructions': "you're a researcher, answer my question briefly, based on your research",
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Researcher',
 'object': 'assistant',
 'tools': []}

# creating a thread

In [16]:
from openai.types.beta import Thread

thread : Thread = client.beta.threads.create(
    
)

show_json(thread)


{'id': 'thread_jh2DlJAq69DY79s0Bsb68bou',
 'created_at': 1706244325,
 'metadata': {},
 'object': 'thread'}

# Creating a message 

In [17]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = " I need some research about how Pakistan came into existenc"

)

show_json(message)



{'id': 'msg_rMDWCmCiRJ6GJOyGhpcazx6N',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': ' I need some research about how Pakistan came into existenc'},
   'type': 'text'}],
 'created_at': 1706244333,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_jh2DlJAq69DY79s0Bsb68bou'}

# Run the assistant

In [18]:
from openai.types.beta.threads.run import Run

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = My_assistant.id,

)

show_json (run)

{'id': 'run_g4R3608vAQhhiw32TqRWkUOG',
 'assistant_id': 'asst_G3EEKDejKTKqiuQUIao3EwXe',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1706244338,
 'expires_at': 1706244938,
 'failed_at': None,
 'file_ids': [],
 'instructions': "you're a researcher, answer my question briefly, based on your research",
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_jh2DlJAq69DY79s0Bsb68bou',
 'tools': [],
 'usage': None}

# Retreival 

In [19]:
#using a while loop so it keeps retreiving until the task is done

import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = My_assistant.id,
        
        )
        time.sleep(0.5)
    return run

        

In [20]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_C0QaDxAGWcdrjjtnxugkmDOM',
   'assistant_id': 'asst_G3EEKDejKTKqiuQUIao3EwXe',
   'content': [{'text': {'annotations': [],
      'value': 'Pakistan came into existence on August 14, 1947, when the Indian Independence Act 1947 was passed by the British Parliament, and the partition of British India occurred, creating the separate nations of India and Pakistan. The creation of Pakistan was based on the demand for a separate Muslim-majority state, and was led by Muhammad Ali Jinnah and the All-India Muslim League.'},
     'type': 'text'}],
   'created_at': 1706244339,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_g4R3608vAQhhiw32TqRWkUOG',
   'thread_id': 'thread_jh2DlJAq69DY79s0Bsb68bou'},
  {'id': 'msg_rMDWCmCiRJ6GJOyGhpcazx6N',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': ' I need some research about how Pakistan came into existenc'},
     'type': 'text'}],
   '

# Display the message 

In [23]:
messages: list[ThreadMessage] = client.beta.threads.messages.list(
    thread_id=thread.id
)

for m in reversed(messages.data):
    print( m.role + ":" + m.content[0].text.value)

user: I need some research about how Pakistan came into existenc
assistant:Pakistan came into existence on August 14, 1947, when the Indian Independence Act 1947 was passed by the British Parliament, and the partition of British India occurred, creating the separate nations of India and Pakistan. The creation of Pakistan was based on the demand for a separate Muslim-majority state, and was led by Muhammad Ali Jinnah and the All-India Muslim League.
